In [43]:
# 모듈 import
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [72]:
# 데이터 로드
purchase_info = pd.read_pickle("./how_to_make/purchase.pickle")
person_info = pd.read_csv("./dataset/LPOINT_BIG_COMP_01_DEMO.csv", low_memory=False)
goods_info = pd.read_csv("./dataset/LPOINT_BIG_COMP_04_PD_CLAC.csv", low_memory = False)

In [73]:
# cust별로 id 부여
person_info['user_id'] = [i for i in range(1,29914)]

In [83]:
# 사용자가 구매한 품목의 합을 구한 데이터
purchase = purchase_info[['cust','pd_c', 'buy_ct','pd_nm', 'clac_mcls_nm']]
purchase = pd.merge(left=purchase, right=person_info, how='inner', on='cust')[['pd_c', 'buy_ct','pd_nm', 'user_id','clac_mcls_nm']]
rating_data = purchase.groupby(['user_id','pd_c']).sum().reset_index()
rating_data

In [80]:
# 물품 데이터
goods_data = goods_info[['pd_c','pd_nm','clac_mcls_nm']]
goods_data

In [84]:
# 데이터 합침
user_goods_rating = pd.merge(rating_data, goods_data, on='pd_c')
user_goods_rating

In [86]:
# 아이템 기반 필터링이므로 index는 품목, columns는 user_id
goods_user_rating = user_goods_rating.pivot_table('buy_ct', index='pd_nm', columns='user_id')
goods_user_rating.fillna(0,inplace=True)
goods_user_rating

In [91]:
# 코사인 유사도 구함
item_based_collabor = cosine_similarity(goods_user_rating)
item_based_collabor = pd.DataFrame(data = item_based_collabor, index=goods_user_rating.index, columns = goods_user_rating.index)
item_based_collabor

array([[1.        , 0.04279799, 0.01503699, ..., 0.        , 0.        ,
        0.        ],
       [0.04279799, 1.        , 0.01593131, ..., 0.02464108, 0.00278621,
        0.        ],
       [0.01503699, 0.01593131, 1.        , ..., 0.02896965, 0.00496995,
        0.00517288],
       ...,
       [0.        , 0.02464108, 0.02896965, ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.00278621, 0.00496995, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.00517288, ..., 0.        , 0.        ,
        1.        ]])

In [100]:
# 추천해주는 함수
def get_item_based_collabor(goods_name):
    return item_based_collabor[goods_name].sort_values(ascending=False)[:6]

In [105]:
get_item_based_collabor('남성향수')

pd_nm
남성향수         1.000000
립스틱/립라이너     0.164011
여성향수         0.109969
수입소고기선물세트    0.092188
스포츠시계        0.089250
선크림류         0.069419
Name: 남성향수, dtype: float64